# Project Sidewalk - get crossing features

In [3]:
# !pip install folium

In [4]:
# !pip install geopandas

In [1]:
import json
import requests
import geopandas as gpd
import folium
import plot_utils

### Settings

In [2]:
CRS = 'epsg:28992'
CRS_map = 'epsg:4326'

## Get project sidewalk data

In [3]:
# Get correct URL
base_url = "https://sidewalk-amsterdam.cs.washington.edu/v2/access/attributesWithLabels?lat1={}&lng1={}&lat2={}&lng2={}" 
amsterdam_bbox_coords = (52.265, 4.73, 52.435, 5.07)
url = base_url.format(*amsterdam_bbox_coords)

In [4]:
# Retrieve data
project_sidewalk_labels = requests.get(url.format(*amsterdam_bbox_coords)).json()

### Create sidewalk data geodataframe

In [12]:
p_sw_gdf = gpd.GeoDataFrame.from_features(project_sidewalk_labels['features'], crs=CRS_map)
p_sw_gdf = p_sw_gdf.to_crs(CRS)
p_sw_gdf.head()

,geometry,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,label_id,gsv_panorama_id,...,image_capture_date,label_date,label_severity,label_is_temporary,agree_count,disagree_count,notsure_count,label_tags,label_description,user_id
0,POINT (126149.896 481976.234),407162,CurbRamp,16735,13886618,E-buurt,1.0,False,31012,hwnYIzNNM0YvnFoE6OOLIg,...,2022-08,2023-04-25 11:30:47.802,1.0,False,0,0,0,[],None,12f7f7b3-ed78-44e9-b92b-23aa57c41772
1,POINT (126149.896 481976.234),407162,CurbRamp,16735,13886618,E-buurt,1.0,False,31009,hwnYIzNNM0YvnFoE6OOLIg,...,2022-08,2023-04-25 11:30:09.611,1.0,False,0,0,0,[],None,12f7f7b3-ed78-44e9-b92b-23aa57c41772
2,POINT (126153.214 481982.157),407172,CurbRamp,16725,13886618,E-buurt,1.0,False,31010,hwnYIzNNM0YvnFoE6OOLIg,...,2022-08,2023-04-25 11:30:27.039,1.0,False,0,0,0,[],None,12f7f7b3-ed78-44e9-b92b-23aa57c41772
3,POINT (126158.336 481974.063),407169,CurbRamp,16725,13886618,E-buurt,1.0,False,31013,hwnYIzNNM0YvnFoE6OOLIg,...,2022-08,2023-04-25 11:31:00.532,1.0,False,0,0,0,[],None,12f7f7b3-ed78-44e9-b92b-23aa57c41772
4,POINT (126166.297 481979.109),407171,CurbRamp,16727,13886618,E-buurt,2.0,False,31014,G7N7BtF_A-g07Cn2G77Zeg,...,2022-08,2023-04-25 11:31:11.403,1.0,False,0,0,0,[],None,12f7f7b3-ed78-44e9-b92b-23aa57c41772


In [14]:
p_sw_gdf.to_csv("/home/azureuser/cloudfiles/code/Users/l.vanoorschot/Project sidewalk/saved dataframes/psw Amsterdam df", index=False)

### Get mask of area

In [6]:
# Select area granularity (buurten/wijken/ggwgebieden/stadsdelen)
area_choice = 'wijken'

# Get area data
area_url = "https://api.data.amsterdam.nl/v1/gebieden/" + area_choice + "/?_format=geojson" 
gdf_area_raw = gpd.read_file(area_url)

# Select columns of geodataframe
gdf_area = gdf_area_raw[['geometry', 'code', 'naam']]

# Adjust CRS
gdf_area = gdf_area.to_crs(CRS)
gdf_area.head(2)

,geometry,code,naam
0,"POLYGON ((129824.496 481758.331, 129749.392 48...",SA,Driemond
1,"POLYGON ((128721.759 480387.302, 128705.327 48...",TK,Nellestein


### Extract sidewalk data in polygon of area

In [7]:
# Function to get psw data from certain wijk

def get_psw_df(wijk):
    polygon = gdf_area[gdf_area['naam'] == wijk].iloc[0]['geometry']
    sw_data_in_area_mask = polygon.contains(p_sw_gdf['geometry'])
    p_sw_area_gdf = p_sw_gdf.loc[sw_data_in_area_mask]

    return p_sw_area_gdf



In [10]:
Osdorp_df = get_psw_df("Osdorp-Midden")
Venserpolder_df = get_psw_df("Venserpolder")

In [12]:
Osdorp_df.to_csv("/home/azureuser/cloudfiles/code/Users/l.vanoorschot/Project sidewalk/saved dataframes/psw Osdorp-Midden df", index=False)

### Inspect data

In [16]:
# # set True for satellite background, False for regular background
# satellite = True

# # Set center of map
# area_coords = [52.35708498470423, 4.79509846197741]

# # Create Folium map
# map = folium.Map(
#     location=area_coords, 
#     tiles=plot_utils.generate_map_params(satellite=satellite),
#     min_zoom=10, max_zoom=25, zoom_start=15,
#     zoom_control=True, control_scale=True, control=False
#     )

# # Add features
# feature_names = p_sw_area_gdf.columns.tolist()
# feature_names.remove('geometry')
# tooltip = plot_utils.gen_tooltip(feature_names, feature_names)
# folium.GeoJson(p_sw_area_gdf, tooltip=tooltip, style_function=lambda x: {"color": "orange", "weight": 10}).add_to(map)

# map